## This week

In [1]:
# 읽기
import pandas as pd
df = pd.read_excel('이번주.xlsx',skiprows = range(0,2))
df = df.drop(0)
# 재심의 제거
df = df.dropna(subset=['결제금액'])
# 반려수 계산 + 결과통보된것만 남겨놓기 
rejected = len(df.loc[df['심의상태']=='반려'])
df = df.loc[df['심의상태']=='결과통보']

### 이번 주 금액
price = df.groupby(['광고매체']).sum()
pr = price.T
pr = pr/1000
pr = pr.astype('int32')

##### 각 항목들 수 

media = ["패키지(표시포함)","인쇄물", "인터넷", "신문", "잡지", "기타(QR 코드 등)", "홈쇼핑", "유선방송(TV, 케이블 TV)", "라디오", "동영상"]
media_detail = ["pack", "printouts", "internet", "newsp", "magaz", "others", "home", "tv", "radio", "youtube"]

p_dict={}
for l in range(len(media)): 
    p_dict[media_detail[l]] = len(df[(df["광고매체"]== media[l] ) & (df["심의결과"]=='승인') ])
      
c_dict={}
for m in range(len(media)): 
    c_dict[media_detail[m]] = len(df[(df["광고매체"]== media[m] ) & (df["심의결과"]=='수정적합') ])
    
f_dict={}
for n in range(len(media)): 
    f_dict[media_detail[n]] = len(df[(df["광고매체"]== media[n] ) & (df["심의결과"]=='기각') ])


In [3]:

# 이번주 결과나간 수 세기
report = pd.DataFrame(
    {" " : ['적합', '수정적합', '부적합', '개수총합'],
 
     "패키지(표시포함)" : [p_dict['pack'], c_dict['pack'], f_dict['pack'], 'n'],
     "인쇄물" : [p_dict['printouts'], c_dict['printouts'], f_dict['printouts'], 'n'],
     "인터넷" : [p_dict['internet'], c_dict['internet'], f_dict['internet'],'n'],
     "신문" : [p_dict['newsp'], c_dict['newsp'], f_dict['newsp'], 'n'],
     "잡지" : [p_dict['magaz'], c_dict['magaz'], f_dict['magaz'], 'n'],
     "기타(QR 코드 등)" : [p_dict['others'], c_dict['others'], f_dict['others'], 'n'],
     "홈쇼핑" : [p_dict['home'], c_dict['home'], f_dict['home'], 'n'],
     "유선방송(TV, 케이블 TV)" : [p_dict['tv'], c_dict['tv'], f_dict['tv'], 'n'],
     "라디오" : [p_dict['radio'], c_dict['radio'], f_dict['radio'], 'n'],
     "동영상" : [p_dict['youtube'], c_dict['youtube'], f_dict['youtube'], 'n'],
     "각각 합" : ['n','n','n','n']
    })
    
# 금액합 row merge               
report.loc[4] = pr.loc['결제금액']
report.iloc[4,0] = '금액합'

# 각각 합 column 
for o in range(5):
    report.iloc[o,11] = report.iloc[o, 1:11].sum(axis=0)

# int -> str
report = report.astype('str').replace('\.0', '', regex=True)
report = report.astype('str').replace('nan', '0', regex=True)

# 개수총합 구하기
for yy in range(11):
    report.iloc[3,yy+1] = report.iloc[0:3,yy+1].astype('int32').sum()


# print()
# print(f'이번주 반려된 건: {rejected}') 




In [4]:
##### 클립보드에 이번주 주간업무보고 복사하기 #####
wk = report.T.reset_index(drop=True)
wk = wk.drop([0,1,2],axis=1)
wk = wk.drop([0],axis=0)
wk.reset_index(drop=True)
wk.to_clipboard(index=False, header=None)


## Next week

In [5]:
## 다음주 읽기 (재심의제외)
import pandas as pd
df2 = pd.read_excel('다음주.xlsx',skiprows = range(0,2))
df2 = df2.drop(0)
df2 = df2.dropna(subset=['결제금액'])


## 다음주 각 항목들 수
n_dict={}
for q in range(len(media)): 
    n_dict[media_detail[q]] = len(df2[(df2["광고매체"]== media[q] ) ])

## 다음주 금액
price2 = df2.groupby(['광고매체']).sum()


In [6]:
# 다음주 결과나간 수 세기
report2 = pd.DataFrame(
    {" " : ['수'],
     "패키지(표시포함)" : n_dict['pack'],
     "인쇄물" : n_dict['printouts'],
     "인터넷" : n_dict['internet'],
     "신문" : n_dict['newsp'],
     "잡지" : n_dict['magaz'],
     "기타(QR 코드 등)" : n_dict['others'],
     "홈쇼핑" : n_dict['home'],
     "유선방송(TV, 케이블 TV)" : n_dict['tv'],
     "라디오" : n_dict['radio'],
     "동영상" : n_dict['youtube'],
         }
)

# 금액 merge

report2.loc[1] = price2['결제금액']/1000
report2.iloc[1,0] = '금액'


# ->int
report2 = report2.astype('str').replace('\.0', '', regex=True)
report2 = report2.astype('str').replace('nan', '0', regex=True)

# 개수총합 구하기
report2['합'] = report2.iloc[:,1:11].astype('int32').sum(axis=1)

##### 클립보드에 이번주 주간업무보고 복사하기 #####

wk2 = report2.T.reset_index(drop=True)
wk2 = wk2.drop([0],axis=0)
wk2.to_clipboard(index=False, header=None)



## This month/year

In [7]:
# 읽기
import pandas as pd
df3 = pd.read_excel('여러달.xlsx',skiprows = range(0,2))
df3 = df3.drop(0)

# 반려수 계산 
rejected = len(df3.loc[df3['심의상태']=='반려'])


# 재심의 제거, 반려, 보류취소도 제거해야함, 
df3 = df3.dropna(subset=['결제금액'])
df3 = df3[~(df3['심의상태']=='반려')]
df3 = df3[~(df3['심의상태']=='보류취소')]


## 각 항목들 수
m_dict={}
for r in range(len(media)): 
    m_dict[media_detail[r]] = len(df3[(df3["광고매체"]== media[r] ) ])

## 금액
price3 = df3.groupby(['광고매체']).sum()


In [8]:
# 결과나간 수 세기
report3 = pd.DataFrame(
    {" " : ['수'],
     "패키지(표시포함)" : m_dict['pack'],
     "인쇄물" : m_dict['printouts'],
     "인터넷" : m_dict['internet'],
     "신문" : m_dict['newsp'],
     "잡지" : m_dict['magaz'],
     "기타(QR 코드 등)" : m_dict['others'],
     "홈쇼핑" : m_dict['home'],
     "유선방송(TV, 케이블 TV)" : m_dict['tv'],
     "라디오" : m_dict['radio'],
     "동영상" : m_dict['youtube'],
         }
)

# 금액 merge

report3.loc[1] = price3['결제금액']/1000
report3.iloc[1,0] = '금액'


# ->int
report3 = report3.astype('str').replace('\.0', '', regex=True)
report3 = report3.astype('str').replace('nan', '0', regex=True)

# 개수총합 구하기
report3['합'] = report3.iloc[:,1:11].astype('int32').sum(axis=1)

##### 클립보드에 주간업무보고 복사하기 #####

wk3 = report3.T.reset_index(drop=True)
wk3 = wk3.drop([0],axis=0)
wk3.to_clipboard(index=False, header=None)

# print('총 반려된 건',end=':') 
# print(rejected)

